<a href="https://colab.research.google.com/github/marianobonelli/analisis_elecciones_nacionales/blob/main/analisis_elecciones_nacionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mariano F. Bonelli

[<img src="https://camo.githubusercontent.com/dd9207aae8b652b023f5cf40711ad4536a4f9b41ca5136648b6c7dda52421da2/68747470733a2f2f696d672e736869656c64732e696f2f62616467652f4d792d4c696e6b6564496e2d626c7565" alt="LinkedIn Badge" data-canonical-src="https://img.shields.io/badge/My-LinkedIn-blue" style="max-width: 100%;">](https://www.linkedin.com/in/mariano-francisco-bonelli/)



---



In [ ]:
# @title Busqueda de los datos
!git clone https://github.com/marianobonelli/analisis_elecciones_nacionales.git



---



In [ ]:
# @title Tabla Resumen
import json
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

def generar_df(env,scope):
    # Leer el archivo JSON
    with open(f'/content/analisis_elecciones_nacionales/assets/{env}/{env}_{scope}.json', 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Obtener el valor "level" más alto si hay más de uno
    max_level = max([item.get('level', 0) for item in data['mapa']])

    # Extraer la información relevante en listas
    scopes_names = []
    partidos_names = []
    votos_values = []
    codigo_ids = []  # Para almacenar los codigo

    for item in data['mapa']:
        if item.get('level', 0) == max_level:  # Filtrar basado en el nivel más alto
            for scope in item['scopes']:
                scope_id = scope['codigo']  # Extraer el codigo
                for partido in scope['partidos']:
                    scopes_names.append(scope['name'])
                    partidos_names.append(partido['name'])
                    votos_values.append(partido['votos'])
                    codigo_ids.append(scope_id)  # Agregar el scopeId a la lista
                # Agregando los votos en blanco como un "partido"
                scopes_names.append(scope['name'])
                partidos_names.append('BLANCOS')
                votos_values.append(scope['blancos'])
                codigo_ids.append(scope_id)  # Agregar el scopeId a la lista

    # Crear un DataFrame a partir de las listas
    df = pd.DataFrame({
        'ID': codigo_ids,
        'Scope': scopes_names,
        'Partido': partidos_names,
        'Votos': votos_values
    })

    # Reestructurar el DataFrame para tener 'Scope' e 'ID' como índices y 'Partido' como columnas
    df = df.pivot(index=['Scope', 'ID'], columns='Partido', values='Votos')

    return df

# Esta función se llama cuando se cambia la selección en los desplegables
def on_change(change):
    clear_output(wait=True)
    display(env_dropdown, scope_dropdown)
    df = generar_df(env_dropdown.value, scope_dropdown.value)
    display(df)

# Leer el archivo JSON
with open('/content/analisis_elecciones_nacionales/assets/indice.json', 'r', encoding='utf-8') as file:
    indice = json.load(file)

# Creación de los desplegables
env_dropdown = widgets.Dropdown(options=['PASO', 'GENERALES'], description='Env:')
scope_dropdown = widgets.Dropdown(options=indice.keys(), description='Scope:')  # Cambia esto según tus necesidades.

# Establecer función a llamar cuando se cambie la selección
env_dropdown.observe(on_change, names='value')
scope_dropdown.observe(on_change, names='value')

# Mostrar los desplegables
display(env_dropdown, scope_dropdown)

# Mostrar el DataFrame inicial
df = generar_df('PASO', 'Buenos Aires')
display(df)



---



In [ ]:
# @title Mapas de Calor
import numpy as np
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

def generar_df(env, scope):
    with open(f'/content/analisis_elecciones_nacionales/assets/{env}/{env}_{scope}.json', 'r', encoding='utf-8') as file:
        data = json.load(file)

    max_level = max([item.get('level', 0) for item in data['mapa']])

    scopes_names = []
    partidos_names = []
    votos_values = []
    codigo_ids = []

    for item in data['mapa']:
        if item.get('level', 0) == max_level:
            for scope in item['scopes']:
                scope_id = scope['codigo']
                for partido in scope['partidos']:
                    scopes_names.append(scope['name'][:13])
                    partidos_names.append(partido['name'][:13])
                    votos_values.append(partido['votos'])
                    codigo_ids.append(scope_id)
                scopes_names.append(scope['name'][:13])
                partidos_names.append('BLANCOS')
                votos_values.append(scope['blancos'])
                codigo_ids.append(scope_id)

    df = pd.DataFrame({
        'ID': codigo_ids,
        'Scope': scopes_names,
        'Partido': partidos_names,
        'Votos': votos_values
    })

    df = df.pivot(index=['Scope', 'ID'], columns='Partido', values='Votos')
    df = df.fillna(0).astype(int)

    # Usar solo 'Scope' como índice
    df.index = df.index.get_level_values('Scope')

    return df


def agregar_total(df):
    df['TOTAL'] = df.sum(axis=1)
    df.loc['TOTAL'] = df.sum(axis=0)
    return df

def mostrar_heatmap_votos(df):
    df = agregar_total(df.copy())
    fmt = ','
    vmax = np.nanmax(df.drop('TOTAL', errors='ignore').values)  # Valor máximo sin considerar totales
    figsize = (max(10, len(df.columns)*0.8), max(8, len(df.index)*0.3))

    plt.figure(figsize=figsize)
    sns.heatmap(df, cmap="YlGnBu", annot=True, fmt=fmt, linewidths=.5, vmax=vmax, cbar=False, alpha=0.7)
    plt.title("Votos")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

def mostrar_heatmap_porcentaje_total(df):
    total_votos = df.values.sum()
    df_percentage = (df / total_votos) * 100

    # Calcular columna TOTAL
    df_percentage['TOTAL'] = (df.sum(axis=1) / total_votos) * 100
    df_percentage.loc['TOTAL'] = df_percentage.sum(axis=0)

    plt.figure(figsize=(max(10, len(df.columns)*0.8), max(8, len(df.index)*0.3)))
    sns.heatmap(df_percentage, cmap="YlGnBu", annot=True, fmt='.2f', linewidths=.5, vmax=50, cbar=False, alpha=0.7)
    plt.title("Porcentaje sobre el total")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    return df_percentage

def mostrar_heatmap_porcentaje_partido(df, total_col):
    total_by_party = df.sum(axis=0)
    df_percentage = (df.divide(total_by_party, axis=1)) * 100

    # Copiar la columna TOTAL desde el DataFrame de Porcentaje sobre el total
    df_percentage['TOTAL'] = total_col
    df_percentage.loc['TOTAL'] = df_percentage.sum(axis=0)

    plt.figure(figsize=(max(10, len(df.columns)*0.8), max(8, len(df.index)*0.3)))
    sns.heatmap(df_percentage, cmap="YlGnBu", annot=True, fmt='.2f', linewidths=.5, vmax=50, cbar=False, alpha=0.7)
    plt.title("Porcentaje por partido")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()


def calcular_porcentaje_total(df):
    total_votos = df.values.sum()
    df_percentage = (df / total_votos) * 100
    df_percentage['TOTAL'] = (df.sum(axis=1) / total_votos) * 100
    df_percentage.loc['TOTAL'] = df_percentage.sum(axis=0)
    return df_percentage

def on_change(change):
    clear_output(wait=True)
    display(env_dropdown, scope_dropdown, graph_dropdown)

    df = generar_df(env_dropdown.value, scope_dropdown.value)
    df_percentage_total = calcular_porcentaje_total(df)

    if graph_dropdown.value == "Votos":
        mostrar_heatmap_votos(df)
    elif graph_dropdown.value == "Porcentaje Total":
        plt.figure(figsize=(max(10, len(df.columns)*0.8), max(8, len(df.index)*0.3)))
        sns.heatmap(df_percentage_total, cmap="YlGnBu", annot=True, fmt='.2f', linewidths=.5, vmax=50, cbar=False, alpha=0.7)
        plt.title("Porcentaje sobre el total")
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.show()
    elif graph_dropdown.value == "Porcentaje Partido":
        mostrar_heatmap_porcentaje_partido(df, df_percentage_total['TOTAL'])

# Definimos los desplegables
env_dropdown = widgets.Dropdown(options=['GENERALES', 'PASO'], description='Env:')
scope_dropdown = widgets.Dropdown(options=indice.keys(), description='Scope:')
graph_dropdown = widgets.Dropdown(
    options=['Votos', 'Porcentaje Total', 'Porcentaje Partido'],
    description='Gráfico:',
    value='Votos'
)

# Observamos los cambios en los desplegables para actualizar las visualizaciones
env_dropdown.observe(on_change, names='value')
scope_dropdown.observe(on_change, names='value')
graph_dropdown.observe(on_change, names='value')

# Mostramos los desplegables
display(env_dropdown, scope_dropdown, graph_dropdown)

# Mostramos el gráfico por defecto para "PASO", "Buenos Aires" y "Votos"
df = generar_df('GENERALES', 'Buenos Aires')
mostrar_heatmap_votos(df)